In [ ]:
# --------------------------------------------------------------
# BANK CUSTOMER CHURN PREDICTION (Complete Mini Project)
# --------------------------------------------------------------
# Problem Statement:
# ...
# Tasks Covered:
# 1. Read the dataset
# 2. Distinguish feature and target set and split data
# 3. Normalize the data
# 4. Build & Train ANN Model + Implement improvements
# 5. Print Accuracy Score & Confusion Matrix
# --------------------------------------------------------------


# ------------------------------------------------------------
# BANK CUSTOMER CHURN PREDICTION USING ARTIFICIAL NEURAL NETWORK (ANN)
# ------------------------------------------------------------
# Aim: Predict whether a bank customer will leave the bank (Churn = 1) or stay (0)
# Steps: Load → Preprocess → Train-Test Split → Normalize → Build ANN → Evaluate
# ------------------------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv("Churn_Modelling.csv")
print("Dataset Shape:", df.shape)

# Display first few rows
print(df.head())


# -----------------------------
# 1. PRE-PROCESSING
# -----------------------------
'''In this step, we separate the independent features (X) and the target variable (y).
Columns like RowNumber, CustomerId, and Surname do not help in prediction and are removed.
Categorical columns such as Geography and Gender are converted into numeric format using One-Hot Encoding.'''

# Select useful features and target
X = df[['CreditScore','Geography','Gender','Age','Tenure','Balance',
        'NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]
y = df['Exited']   # Target → 1 = Churn, 0 = Not Churn

# Convert categorical columns into numeric values
X = pd.get_dummies(X, drop_first=True)

# -----------------------------
# 2. TRAIN-TEST SPLIT
# -----------------------------
'''train_test_split() divides the dataset into training and testing parts.
X_train and y_train will be used to train the ANN model, 
while X_test and y_test will be used to check how well the model performs on unseen data.
test_size=0.2 means 80% training and 20% testing. random_state=42 ensures reproducibility.'''

# Split data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 3. DATA NORMALIZATION
# -----------------------------
'''ANN models perform better when numerical features are on a similar scale.
StandardScaler transforms all feature values so that they have mean=0 and std=1, 
leading to faster learning and better model accuracy.'''

# Normalize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -----------------------------
# 4. MODEL TRAINING (ANN)
# -----------------------------
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

'''This block creates and trains the Artificial Neural Network model.

Sequential() initializes the ANN as a sequence of layers. Dense() is used to add fully connected layers, 
where each neuron receives input from all neurons of the previous layer. The first Dense layer specifies 
input_dim=X_train.shape[1] to match the number of input features.

ReLU activation allows the ANN to learn non-linear patterns, while Dropout(0.2) helps prevent overfitting 
by randomly disabling 20% of neurons during training. The output layer uses a sigmoid activation function 
to return values between 0 and 1 for binary classification (Churn/Not Churn).

The compile() function configures the model with the Adam optimizer and binary_crossentropy loss function, 
both suitable for binary classification tasks. Finally, fit() trains the ANN using the training data for 
20 epochs with a batch size of 32 and uses 20% of training data for validation during training.'''


# Initialize ANN Model
model = Sequential()

# Input + Hidden Layer 1
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))

# Hidden Layer 2
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(1, activation='sigmoid'))  # Sigmoid outputs probability between 0 and 1

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


# -----------------------------
# 5. MODEL EVALUATION
# -----------------------------
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

'''This block evaluates the performance of the trained ANN model.

The predict() function generates probability values for each test sample, so a threshold of 0.5 is applied 
to convert them into binary class labels (0 = Not Churn, 1 = Churn). The accuracy_score() function measures 
the percentage of correct predictions made by the model.

confusion_matrix() returns a matrix showing True Positives, True Negatives, False Positives, and False Negatives, 
helping to analyze model performance for each class. classification_report() provides a detailed summary containing 
precision, recall, F1-score, and support, giving deeper insight into how well the model performs across both classes.'''


# Predict on test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Accuracy Score
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

